In [ ]:
%pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 400.6 kB/s eta 0:00:0000:0100:02


In [ ]:
%pip install stable_baselines3

  Using cached stable_baselines3-2.4.0-py3-none-any.whl.metadata (4.5 kB)
Using cached stable_baselines3-2.4.0-py3-none-any.whl (183 kB)


In [3]:
import os
import subprocess

if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Other imports and helper functions
import time
import itertools
import numpy as np

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()

In [15]:
import gymnasium as gym
from gymnasium import spaces
import torch
import numpy as np
import mujoco
import glfw

In [16]:
def define_model(l1, l2, k):
    xml = f"""
    <mujoco model="robot">
  <option apirate="2000"/>
  <default>
    <default class="robot/"/>
    </default>
  <worldbody>
    <body name="body_chassis_" pos="0 0 1" euler="0 0 0">
      <camera name="camera_track_chassis_" class="robot/" mode="track" pos="0 -3 1" zaxis="0 -1 0.5"/>
      <geom name="geom_chassis_" class="robot/" type="box" size="0.22 0.16 0.10000000000000001" rgba="0 0.29999999999999999 0.5 1" mass="15"/>
      <site name="right_site_chassis_" class="robot/" size="9.9999999999999995e-07 9.9999999999999995e-07 9.9999999999999995e-07" pos="0 -0.16 0"/>
      <body pos="0 0 0" name="right_upper_link/">
        <body name="right_upper_link/body_link_right_upper">
          <geom name="right_upper_link/geom_link_right_upper" class="robot/" type="box" size="0.025000000000000001 0.025000000000000001 {l1}" rgba="0 0.29999999999999999 0.5 1" mass="2" pos="0 -0.025000000000000001 -0.20000000000000001"/>
          <joint name="right_upper_link/joint_link_right_upper" class="robot/" type="hinge" axis="0 1 0" ref="20.0" armature="0.010901249999999999" damping="0.073900000000000001" frictionloss="0.13339999999999999" range="-60 60"/>
          <site name="right_upper_link/link_out_site_right_upper" class="robot/" size="9.9999999999999995e-07 9.9999999999999995e-07 9.9999999999999995e-07" pos="0 -0.050000000000000003 -0.40000000000000002"/>
          <body pos="0 -0.050000000000000003 {-1.7*l1}" name="right_upper_link/right_down_link/">
            <body name="right_upper_link/right_down_link/body_link_right_down">
              <geom name="right_upper_link/right_down_link/geom_link_right_down" class="robot/" type="box" size="0.025000000000000001 0.025000000000000001 {l2}" rgba="0 0.29999999999999999 0.5 1" mass="2" pos="0 -0.025000000000000001 -0.20000000000000001"/>
              <joint name="right_upper_link/right_down_link/joint_link_right_down" class="robot/" type="hinge" axis="0 1 0" ref="100.0" armature="0.010901249999999999" damping="0.023900000000000001" range="-70 70" frictionloss="0.13339999999999999" stiffness="{k}"/>
              <site name="right_upper_link/right_down_link/link_out_site_right_down" class="robot/" size="9.9999999999999995e-07 9.9999999999999995e-07 9.9999999999999995e-07" pos="0 -0.050000000000000003 -0.40000000000000002"/>
              <body pos="0 -0.050000000000000003 -0.40000000000000002" name="right_upper_link/right_down_link/right_tire/">
              </body>
            </body>
          </body>
        </body>
      </body>
      <joint name="free_floating_chassis_" class="robot/" type="slide" axis="0 0 1" ref="0.0"/>
      <site name="imu" class="robot/" size="9.9999999999999995e-07 9.9999999999999995e-07 9.9999999999999995e-07" pos="0 0 0"/>
    </body>
    <light name="spotlight_0" class="robot/" mode="targetbodycom" target="body_chassis_" pos="0.25 0.19 0.15000000000000002"/>
    <light name="spotlight_1" class="robot/" mode="targetbodycom" target="body_chassis_" pos="0.25 -0.19 0.15000000000000002"/>
    <light name="spotlight_2" class="robot/" mode="targetbodycom" target="body_chassis_" pos="-0.25 0.19 0.15000000000000002"/>
    <light name="spotlight_3" class="robot/" mode="targetbodycom" target="body_chassis_" pos="-0.25 -0.19 0.15000000000000002"/>
  </worldbody>
  <actuator>
    <motor name="my_motor" joint="right_upper_link/joint_link_right_upper" gear="100" forcelimited="true" forcerange="-10 10"/>
    <motor name="my_motor1" joint="right_upper_link/right_down_link/joint_link_right_down" forcelimited="true" forcerange="-10 10" gear="500"/>
  </actuator>
  <sensor>
    <jointpos name="rbody_chassi_/pos" joint="free_floating_chassis_"/>
    <jointvel name="rbody_chassi_/vel" joint="free_floating_chassis_"/>
    <jointpos name="right_upper_link/pos" joint="right_upper_link/joint_link_right_upper"/>
    <jointvel name="right_upper_link/vel" joint="right_upper_link/joint_link_right_upper"/>
    <jointpos name="right_upper_link/right_down_link/pos" joint="right_upper_link/right_down_link/joint_link_right_down"/>
    <jointvel name="right_upper_link/right_down_link/vel" joint="right_upper_link/right_down_link/joint_link_right_down"/>
  </sensor>
  </mujoco>
    """
    file = open("robot.xml", "w")
    file.write(xml)
    file.close()
    return xml

define_model(0.29999999999999999,0.29999999999999999,500.0)

'\n    <mujoco model="robot">\n  <option apirate="2000"/>\n  <default>\n    <default class="robot/"/>\n    </default>\n  <worldbody>\n    <body name="body_chassis_" pos="0 0 1" euler="0 0 0">\n      <camera name="camera_track_chassis_" class="robot/" mode="track" pos="0 -3 1" zaxis="0 -1 0.5"/>\n      <geom name="geom_chassis_" class="robot/" type="box" size="0.22 0.16 0.10000000000000001" rgba="0 0.29999999999999999 0.5 1" mass="15"/>\n      <site name="right_site_chassis_" class="robot/" size="9.9999999999999995e-07 9.9999999999999995e-07 9.9999999999999995e-07" pos="0 -0.16 0"/>\n      <body pos="0 0 0" name="right_upper_link/">\n        <body name="right_upper_link/body_link_right_upper">\n          <geom name="right_upper_link/geom_link_right_upper" class="robot/" type="box" size="0.025000000000000001 0.025000000000000001 0.3" rgba="0 0.29999999999999999 0.5 1" mass="2" pos="0 -0.025000000000000001 -0.20000000000000001"/>\n          <joint name="right_upper_link/joint_link_right_u

In [ ]:
from pathlib import Path
import enum
from tqdm import tqdm
import math

ModuleNotFoundError: No module named 'torch.hub'

In [18]:
class Resolution(enum.Enum):
  SD = (480, 640)
  HD = (720, 1280)
  UHD = (2160, 3840)


def quartic(t: float) -> float:
  return 0 if abs(t) > 1 else (1 - t**2) ** 2


def blend_coef(t: float, duration: float, std: float) -> float:
  normalised_time = 2 * t / duration - 1
  return quartic(normalised_time / std)


def unit_smooth(normalised_time: float) -> float:
  return 1 - np.cos(normalised_time * 2 * np.pi)


def azimuth(
    time: float, duration: float, total_rotation: float, offset: float
) -> float:
  return offset + unit_smooth(time / duration) * total_rotation

In [21]:
res = Resolution.SD
fps = 60
duration = 10.0
ctrl_rate = 2
ctrl_std = 0.05
total_rot = 60
blend_std = .8


In [22]:
h, w = res.value

In [23]:
# Load model.
model = mujoco.MjModel.from_xml_path("scene.xml")
data = mujoco.MjData(model)

# Make sure offscreen rendering can support the desired resolution.
model.vis.global_.offheight = h
model.vis.global_.offwidth = w

renderer = mujoco.Renderer(model, height=h, width=w)


In [24]:
data.actuator_force

array([0., 0.])

In [40]:
class HopperEnv(gym.Env):
    def __init__(self):
        # Load the MuJoCo model from XML string
        self.model = mujoco.MjModel.from_xml_path("scene.xml")
        self.sim = mujoco.MjData(self.model)

        # Define action and observation spaces
        num_actuators = self.model.nu
        num_observations = 7#2*self.model.nq  # This is just an example; adjust as needed
        self.action_space = spaces.Box(low=-1, high=1, shape=(num_actuators,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_observations,), dtype=np.float32)

        self.timestep = 0



    def step(self, action):
        self.timestep += 1

        # Apply the action to the environment
        self.sim.ctrl[:] = action
        mujoco.mj_step(self.model, self.sim)

        # Get the observation, reward, done, and info
        observation = self._get_observation()
        reward = self._get_reward()
        trunctuated = False
        info = {}

        return observation, reward, trunctuated, info

    def reset(self, seed = None, **kwargs):
        # Set initial conditions
        init_qpos = [0, 20, 100, 0, 0, 0, 0, ...] 
        init_qvel = [0, 0, 0, 0, 0, 0, 0, ...]

        # Reset MuJoCo
        self.sim = mujoco.MjData(self.model)
        mujoco.mj_forward(self.model, self.sim)

        # Get observation 
        obs = self._get_observation()
        mujoco.mj_forward(self.model, self.sim)

        reset_info = {}  # This can be populated with any reset-specific info if needed
        return obs, reset_info


    def _get_observation(self):
      # Joint positions
      qpos = self.sim.qpos
      
      # Joint velocities
      qvel = self.sim.qvel

      contact_forces = np.sum(np.sqrt(np.sum(np.square(np.array(self.sim.cfrc_ext)), axis=1)))
      
      # Concatenate and return as a single observation vector
      observation = np.concatenate([qpos, qvel, [contact_forces]])
      
      return observation


    def _get_reward(self):
        w_f = 1
        w_z = 5
        stance_diff = w_f*np.linalg.norm(self.sim.actuator_force[0]) + w_f*np.linalg.norm(self.sim.actuator_force[1]) + w_z*self.sim.qpos[1]
        return -stance_diff

    def sample_random_action(self):

      # Get action bounds
      action_low = self.action_space.low
      action_high = self.action_space.high
      
      # Sample random action as numpy array
      action = np.random.uniform(low=action_low, high=action_high)
      
      return action

class HopperEnvWrapper(gym.Wrapper):

  def __init__(self, env):
    super().__init__(env)
  
  def reset(self, seed=None, **kwargs):
    obs = self.env.reset()
    return obs
  
  def step(self, action):
    obs, reward, trunctuated, info = self.env.step(action)
    
    # Calculate stance reward
    reward = self.env._get_reward
    
    return obs, reward, trunctuated, info
  
from gymnasium.envs.registration import register

register(
    id='BoxingEnv-v2',
    entry_point='final_boxing:BoxingEnv', 
)
env = gym.make('BoxingEnv-v2')



/home/daniel/sber_lab/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment BoxingEnv-v2 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


ModuleNotFoundError: No module named 'final_boxing'

ModuleNotFoundError: No module named 'torch.autograd'